In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import classificacao

sns.set_style('darkgrid')
%matplotlib inline

In [3]:
df = pd.read_csv('conhecimento.csv')
df['dia'] = df.id_cena.str[-3:].astype(int)

id_cenas = [
    # 'LS82260702016078', 
    'LS82260702017256', # 226_070
    # 'LS82210702016059', 
    'LS82210702017269', # 221_070
    # 'LS52210672002068', 
    'LS82210672017285', # 221_067
    # 'LS82210742016059', 
    'LS82210742018128', # 221_074
]
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena', 'caso']

In [4]:
def gera_teste_modelo_caso1(id_cena):
    teste_original = df[df.id_cena== id_cena].copy().reset_index(drop=True)
    teste = teste_original.copy()
    teste['verifica'] = 0

    # Modelo
    modelo = df[~(df.id_cena==id_cena)].copy()
    return teste, teste_original, modelo

def gera_teste_modelo_caso2(id_cena):
    
    teste_original = df[df.id_cena==id_cena].copy().reset_index(drop=True)
    teste = teste_original.copy()
    teste['verifica'] = 0

    # Modelo
    modelo_inteiro = df[~(df.id_cena==id_cena)].copy()

    modelo_inteiro_1 = modelo_inteiro[modelo_inteiro.verifica==1].copy()
    modelo_inteiro_3_full = modelo_inteiro[modelo_inteiro.verifica==3].copy()

    id3 = np.random.choice(modelo_inteiro_3_full.id, len(modelo_inteiro_1), replace=False) 
    modelo_inteiro_3 = modelo_inteiro_3_full[modelo_inteiro_3_full.id.isin(id3)]

    modelo = pd.concat([modelo_inteiro_1, modelo_inteiro_3]).reset_index(drop=True)

    return teste, teste_original, modelo

def gera_teste_modelo_caso3(id_cena):
    print(id_cena)
    
    dia = int(id_cena[-3:])
    print(df.shape, dia, id_cena)

    teste_original = df[df.id_cena== id_cena].copy().reset_index(drop=True)
    teste = teste_original.copy()
    teste['verifica'] = 0
    print(len(teste_original))

    # Modelo
    op = teste.orb_pto.loc[0]
    print(op)
    modelo = df[(df.dia.isin(range(dia-90, dia+90))) & (df.orb_pto == op)].copy()

    print(len(modelo))
    print(modelo.verifica.value_counts())

    return teste, teste_original, modelo

In [5]:
def get_id_cenas(df, orb_pto):
    return sorted(list(df[df.orb_pto==orb_pto].id_cena.unique()))
def resultado(teste, teste_original, verifica=1):
    r = pd.merge(teste[['id', 'classificacao']], teste_original[['id', 'verifica']], on='id')
    r['v'] = r.classificacao.astype(str) + r.verifica.astype(str)
    
    try:
        x = len(r[(r.verifica==verifica) & (r.classificacao == verifica)])
        v = len(r[r.verifica==verifica])
        p = int((x*100)/v)
    except:
        return [0, 0, 0]
    return [x, v, p]

In [6]:
sumario = pd.DataFrame([], columns=col_sum)
for id_cena in id_cenas:
    teste, teste_original, modelo = gera_teste_modelo_caso1(id_cena)
    y_pred = classificacao.classifica(modelo, teste)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = resultado(y_pred, y_true)
    r.append(id_cena)
    r.append(1)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)
    
    teste, teste_original, modelo = gera_teste_modelo_caso2(id_cena)
    y_pred = classificacao.classifica(modelo, teste)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = resultado(y_pred, y_true)
    r.append(id_cena)
    r.append(2)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)
    
    teste, teste_original, modelo = gera_teste_modelo_caso3(id_cena)
    y_pred = classificacao.classifica(modelo, teste)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = resultado(y_pred, y_true)
    r.append(id_cena)
    r.append(3)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)

    print(r)

Criando classificador RF


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/queimadas/.conda/envs/py3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/queimadas/.con

* Caso 1: Usando todo o conhecimento
* Caso 2: Quantidade de verifica 1 e 3 iguais
* Caso 3: Trimestre historico da orb_pto
> Em todos casos, o dado da própria passagem foi removido do conhecimento. E só esta sendo contabilizado dados de acerto de QUEIMADA

In [ ]:
sumario

In [8]:
sumario.to_csv('resultado_melhores_colunas.csv', index=False)